In [1]:
CORPUS = ['the green car is fast',
 'The blue car has low mileage',
 'Your car has high mileage',
 'I buy green shirt because it fits my blue eyes']
new_doc = ['I want to buy a car']
type(new_doc)

list

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
def bow_generator(corpus, ngram_range=(1,2)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


# build bow vectorizer and get features
bow_vectorizer, bow_features = bow_generator(CORPUS)
features = bow_features.todense()
print (features)

[[0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1
  0 0]
 [0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0
  0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  1 1]
 [1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0
  0 0]]


In [3]:
# print the feature names
feature_names = bow_vectorizer.get_feature_names()
print (feature_names)

['because', 'because it', 'blue', 'blue car', 'blue eyes', 'buy', 'buy green', 'car', 'car has', 'car is', 'eyes', 'fast', 'fits', 'fits my', 'green', 'green car', 'green shirt', 'has', 'has high', 'has low', 'high', 'high mileage', 'is', 'is fast', 'it', 'it fits', 'low', 'low mileage', 'mileage', 'my', 'my blue', 'shirt', 'shirt because', 'the', 'the blue', 'the green', 'your', 'your car']


In [4]:
# extract features from new document using built vectorizer
new_doc_features = bow_vectorizer.transform(new_doc)
new_doc_features = new_doc_features.todense()
print(new_doc_features)

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0]]


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(norm='l2',smooth_idf=True, use_idf=True) #smoothing gives sme weights to terms with zero idf not to ignore them.
tfidf_matrix = transformer.fit_transform(bow_features)
features = np.round(tfidf_matrix.todense(), 2)
 
df_tfidf = pd.DataFrame(data=features, columns=feature_names)
df_tfidf

,because,because it,blue,blue car,blue eyes,buy,buy green,car,car has,car is,...,mileage,my,my blue,shirt,shirt because,the,the blue,the green,your,your car
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.00,0.36,...,0.00,0.00,0.00,0.00,0.00,0.29,0.00,0.36,0.00,0.00
1,0.00,0.00,0.27,0.34,0.00,0.00,0.00,0.22,0.27,0.00,...,0.27,0.00,0.00,0.00,0.00,0.27,0.34,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.24,0.29,0.00,...,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.37,0.37
3,0.25,0.25,0.20,0.00,0.25,0.25,0.25,0.00,0.00,0.00,...,0.00,0.25,0.25,0.25,0.25,0.00,0.00,0.00,0.00,0.00


In [7]:
# tfidf features for new_doc
tfidf_matrix = transformer.fit_transform(new_doc_features)
features_new = np.round(tfidf_matrix.todense(), 2)
 
df_tfidf_new = pd.DataFrame(data=features_new, columns=feature_names)
df_tfidf_new

,because,because it,blue,blue car,blue eyes,buy,buy green,car,car has,car is,...,mileage,my,my blue,shirt,shirt because,the,the blue,the green,your,your car
0,0.0,0.0,0.0,0.0,0.0,0.71,0.0,0.71,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
import gensim # commonly used for semantic analysis, topic modeling and similarity analysis.
import nltk
# tokenize corpora
TOKENIZED_CORPUS = [nltk.word_tokenize(sentence) for sentence in CORPUS]
tokenized_new_doc = [nltk.word_tokenize(sentence) for sentence in new_doc]
# build the word2vec model on our training corpus
model = gensim.models.Word2Vec(TOKENIZED_CORPUS, size=5, window=2, min_count=1)
print (model.wv['green'])
print (model.wv['blue'])

[ 0.07155872  0.05740496 -0.02537698 -0.026161   -0.074492  ]
[ 0.09699327  0.02634738  0.01832688  0.06455417 -0.08762705]


In [11]:
from gensim.models import word2vec
model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)
[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399)] 

KeyboardInterrupt: 

In [3]:

model.most_similar(positive=['robots', 'future', 'humans'], topn=2)
[('autonomous_robots', 0.6404092907905579),
 ('humanoid_robots', 0.6080278754234314)]
 
print("The similarity between 'United_States' " + "and 'Canada':", model.similarity('United_States', 'Canada')) 
print("The similarity between 'United_States' " + "and 'Tunisia':", model.similarity('United_States', 'Tunisia')) 


[['the', 'green', 'car', 'is', 'fast'], ['The', 'blue', 'car', 'has', 'low', 'mileage'], ['Your', 'car', 'has', 'high', 'mileage'], ['I', 'buy', 'green', 'shirt', 'because', 'it', 'fits', 'my', 'blue', 'eyes']]


In [ ]:
#Term that does not fit in
model.doesnt_match("bank debt credit banana".split())
'banana'
 
model.distance("man","woman") # distance = 1- similarity

In [ ]:
from nltk.corpus import brown
sentences = brown.sents()
sentences
 
[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]
 
model = gensim.models.Word2Vec(sentences, min_count=1)
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)
 
[('Eastwick', 0.9702203273773193),
 ('80', 0.9639546871185303),
 ('diameter', 0.9638100266456604)]
 
model.wv.most_similar(positive=['robots', 'future', 'humans'], topn=2)
[('compromise', 0.9315764904022217), ('benefits', 0.9312320947647095)]

In [ ]:
from gensim.test.utils import common_texts
common_texts[0:3]
 
[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system']]

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(common_texts)# we assigned a unique integer id to all words appearing in the corpus with the gensim Dictionary class.
print(dictionary) #each document will be represented by the number of distinct words in the corpus (total vector size))

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)

print(dictionary.token2id) # he mapping between words and their ids
{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}

In [ ]:
corpus = [dictionary.doc2bow(text) for text in common_texts] # each word has an Id and each document has words
corpus

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents
 
[TaggedDocument(words=['human', 'interface', 'computer'], tags=[0]),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=[1]),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=[2]),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=[3]),
 TaggedDocument(words=['user', 'response', 'time'], tags=[4]),
 TaggedDocument(words=['trees'], tags=[5]),
 TaggedDocument(words=['graph', 'trees'], tags=[6]),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=[7]),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=[8])]

In [ ]:
#  building a model
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, dm =1)
#Infer vector for a new document:
vector = model.infer_vector(["system", "response"])
vector
array([-0.03828789,  0.06952108,  0.02473402,  0.01858516, -0.08879752],
      dtype=float32)